In [ ]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('result/places.db')
cursor = conn.cursor()

In [ ]:
cursor.execute("select rowid, mapURL from places")
mapURLs = cursor.fetchall()

# create csv file for mapURLs
with open("mapURLs.csv", "w") as f:
    f.write("rowid,mapURL\n")
    for rowid, mapURL in mapURLs:
        f.write(f"{rowid},{mapURL}\n")



In [ ]:
import sqlite3

conn = sqlite3.connect('result/places.db')
cursor = conn.cursor()

# Delete duplicate rows, keeping the first row (lowest rowid) for each mapURL
cursor.executescript("""
    DELETE FROM places
    WHERE rowid NOT IN (
        SELECT MIN(rowid)
        FROM places
        GROUP BY mapURL
    );
""")

result = cursor.fetchall()
print(result)

conn.commit()
cursor.close()
conn.close()

In [ ]:
# attach 'reviews.db' as review;

# alter table places add column firstReviews;

# update places
# set places.firstReviews = review.firstReviews
# where places.mapURL = review.mapURL;

# detach review;

import sqlite3
conn = sqlite3.connect('result/places.db')

cursor = conn.cursor()

# check rows miss in reviews.db review
script = """
    ATTACH DATABASE 'result/reviews.db' AS review;

-- Update places.firstReviews by exact match after cleaning
UPDATE places
SET firstReviews = (
    SELECT r.firstReviews
    FROM review.reviews r
    WHERE TRIM(r.mapURL) = TRIM(places.mapURL)
    LIMIT 1
);

DETACH DATABASE review;
    """ 
cursor.executescript(script)
print(cursor.fetchall())
cursor.close()

In [ ]:
# attach 'reviews.db' as review;

# alter table places add column firstReviews;

# update places
# set places.firstReviews = review.firstReviews
# where places.mapURL = review.mapURL;

# detach review;

import sqlite3
conn = sqlite3.connect('result/places.db')

cursor = conn.cursor()

# check rows miss in reviews.db review
cursor.execute("select mapURL from places where rowid <= 2")
print(cursor.fetchall())

cursor.close()

In [ ]:
# attach 'reviews.db' as review;

# alter table places add column firstReviews;

# update places
# set places.firstReviews = review.firstReviews
# where places.mapURL = review.mapURL;

# detach review;

import sqlite3
conn = sqlite3.connect('result/reviews.db')

cursor = conn.cursor()

# check rows miss in reviews.db review
cursor.execute("select mapURL from reviews")
result = cursor.fetchall()
print(result)
print(len(result))

cursor.close()

In [ ]:
import sqlite3

conn = sqlite3.connect('result/places.db')
cursor = conn.cursor()

# Use execute() for SELECT queries
script = """
    SELECT COUNT(*) 
    FROM (
        SELECT DISTINCT mapURL 
        FROM places
    );
"""
cursor.execute(script)
result = cursor.fetchall()  # fetch results after execute
print(result)

cursor.close()
conn.close()


In [ ]:
import sqlite3

conn = sqlite3.connect('result/reviews.db')
cursor = conn.cursor()

cursor.execute("SELECT mapURL FROM reviews")
results = cursor.fetchall()

for i, r in enumerate(results):
    if i > 10:
        break
    print(repr(r[0]))  # show hidden characters like \r or \n

cursor.close()
conn.close()

In [ ]:
import sqlite3
import json

conn = sqlite3.connect('result/places.db')
cursor = conn.cursor()

try:
    cursor.execute("ALTER TABLE places ADD COLUMN images TEXT")
except sqlite3.OperationalError:
    pass

cursor.execute("SELECT rowid, imageurl, firstReviews FROM places")
rows = cursor.fetchall()

for row in rows:
    rowid, imageurl, first_reviews_str = row
    image_list = []
    if imageurl:
        image_list.append(imageurl)
    if first_reviews_str:
        try:
            reviews = json.loads(first_reviews_str)
            for review in reviews:
                if 'imageUrls' in review and isinstance(review['imageUrls'], list):
                    image_list.extend(review['imageUrls'])
        except json.JSONDecodeError:
            pass
    images_json = json.dumps(image_list)

    cursor.execute(
        "UPDATE places SET images = ? WHERE rowid = ?",
        (images_json, rowid)
    )

conn.commit()
cursor.close()
conn.close()

print("success")


In [8]:
import re
import json
import sqlite3


def to_fullsize(url):
    # thay mọi phần bắt đầu bằng '=' đến cuối chuỗi thành '=s0'
    return re.sub(r'=[^?]+$', '=s0', url)


conn = sqlite3.connect("result/places.db")
cursor = conn.cursor()
cursor.execute("select rowid, images from places")
result = cursor.fetchall()
print(len(result))

for row in result:
    id, images = row
    ## convert list
    images = json.loads(images)
    images = [to_fullsize(img) for img in images]
    images_json = json.dumps(images)
    cursor.execute("update places set images = ? where rowid = ?", (images_json, id))

519


In [10]:
conn.commit()
cursor.close()
conn.close()